<a href="https://www.kaggle.com/code/annaselinabaniakina/digit-recognition?scriptVersionId=200661129" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
#load datasets
train_data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test_data = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [3]:
#reshape and normalize training data
X_train = train_data.drop(columns='label').values.reshape(-1, 28, 28, 1) / 255.0
y_train = train_data['label']

#normalize test data
test_data = test_data.values.reshape(-1, 28, 28, 1) / 255.0

In [4]:
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class AGI_digit_recognition:
    def __init__(self, input_shape=(28, 28, 1)):
        self.knowledge_base = []  #a simple knowledge base to store experiences
        self.model = self.initialize_learning_model(input_shape)
        self.data_augmentation = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            shear_range=0.1,
            fill_mode='nearest'
        )

    def initialize_learning_model(self, input_shape):
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(10, activation='softmax')
        ])
        model.compile(optimizer='adam', 
                      loss='sparse_categorical_crossentropy', 
                      metrics=['accuracy'])
        return model

    def learn_from_experience(self, input_data, labels):
        #augment data and learn from it
        self.knowledge_base.append((input_data, labels))
        self.update_model(input_data, labels)

    def update_model(self, input_data, labels):
        #fit the model with augmented data
        augmented_data = self.data_augmentation.flow(input_data, labels, batch_size=64)
        self.model.fit(augmented_data, epochs=30)

    def reason(self, new_input):
        #predict using the current model
        predictions = self.model.predict(new_input)
        return np.argmax(predictions, axis=1)

    def adapt(self, new_task_data, new_task_labels):
        self.learn_from_experience(new_task_data, new_task_labels)

#split the training data for validation
X_train_img, X_val_img, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#initialize the AGI model
agi_model = AGI_digit_recognition()

#train the model with the training data
agi_model.learn_from_experience(X_train_img, y_train)

#validate the model on validation data
val_predictions = agi_model.reason(X_val_img)

2024-10-12 21:08:50.078839: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 21:08:50.078976: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 21:08:50.241590: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/30
525/525 [==============================] - 21s 38ms/step - loss: 0.7250 - accuracy: 0.7638
Epoch 2/30
525/525 [==============================] - 20s 38ms/step - loss: 0.2828 - accuracy: 0.9134
Epoch 3/30
525/525 [==============================] - 19s 37ms/step - loss: 0.2185 - accuracy: 0.9343
Epoch 4/30
525/525 [==============================] - 20s 38ms/step - loss: 0.1823 - accuracy: 0.9457
Epoch 5/30
525/525 [==============================] - 19s 37ms/step - loss: 0.1621 - accuracy: 0.9522
Epoch 6/30
525/525 [==============================] - 20s 38ms/step - loss: 0.1473 - accuracy: 0.9560
Epoch 7/30
525/525 [==============================] - 20s 38ms/step - loss: 0.1377 - accuracy: 0.9594
Epoch 8/30
525/525 [==============================] - 20s 37ms/step - loss: 0.1256 - accuracy: 0.9623
Epoch 9/30
525/525 [==============================] - 20s 38ms/step - loss: 0.1205 - accuracy: 0.9646
Epoch 10/30
525/525 [==============================] - 20s 37ms/step - loss: 0.117

In [5]:
#make predictions on the test data
test_predictions = agi_model.reason(test_data)

#prepare the submission DataFrame
submission = pd.DataFrame({
    'ImageId': np.arange(1, len(test_predictions) + 1),
    'Label': test_predictions
})

#save the submission file
submission.to_csv('/kaggle/working/submission.csv', index=False)

875/875 [==============================] - 5s 5ms/step
